# Clasificador K-NN en Spark usando pyspark.dataframes

### Se importan las librerías necesarias

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import random
import math
import sys
import numpy as np 

### Se crea la sesión y config. de Spark

In [2]:
# Build the SparkSession
spark = SparkSession.builder \
    .master("local[6]") \
    .appName("Data exploration URL - KNN") \
    .config("spark.executor.memory", "4gb") \
    .getOrCreate()

sc = spark.sparkContext

In [3]:
sc._conf.getAll()

[('spark.executor.memory', '4gb'),
 ('spark.app.name', 'Data exploration URL - KNN'),
 ('spark.app.id', 'local-1617063923033'),
 ('spark.executor.id', 'driver'),
 ('spark.app.startTime', '1617063922367'),
 ('spark.master', 'local[6]'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.sql.warehouse.dir',
  'file:/home/jsarabia/Documents/IA/Data-exploration-url_svmlight/code/spark-warehouse'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '33385'),
 ('spark.driver.host', 'fedora')]

In [4]:
sc

<SparkContext master=local[6] appName=Data exploration URL - KNN>

### Se cargan los datos al dataframe 

In [16]:
# Load training data
data = spark.read.format("libsvm")\
    .load("../data/url_svmlight/Dimension_100_x_500000.svm")
# Split the data into train and test
seed = random.randrange(500, 1300, 2)
splits = data.randomSplit([0.7, 0.3], 1234)

train = splits[0]
test = splits[1]

### Calcular la distancia euclideana.
#### Summary:
        Se calcula la distancia entre las columnas de dos renglones de un dataset, funciona
        con argumentos provenientes de un renglón de un dataframe de Spark.
#### Args: 
        row1(pyspark.sql.types.Row): Recibe una instancia del dataset
        row2(pyspark.sql.types.Row): Recibe una instancia del dataset

In [6]:
def euclidean_distance(row1, row2):
    distance = 0.0
    for column in range(len(row1[1])):
        distance += pow(row1[1][column] - row2[1][column], 2)
    distance = math.sqrt(distance)
    return distance

### Obtener los vecinos más cercanos.
#### Summary: 
      Se recorre cada renglón del dataframe dado y se calcula la distancia entre cada 
      uno de estos y el renglón de prueba.
      Se crea la lista "distances", la cual almacenar[a las distancias calculadas, 
      posteriormente se ordena de modo ascendente y se almancenan los primeros k-elementos 
      en la lista "neighbors"

#### Args: 
      train(pyspark.sql.dataframe.DataFrame): Recibe el dataframe de entrenamiento
      test_row(pyspark.sql.types.Row): Recibe una instancia o renglón del dataset
      k(int): Número de k-vecinos que se desean obtener

In [7]:
def get_neighbors(train, test_row, k):
    distances = []
    total_train_rows = train.count() + 1
    for train_row in range(1, total_train_rows):
        distance = euclidean_distance(test_row, train.head(train_row)[-1])
        if(distance != 0.0):
           distances.append((train.head(train_row)[-1], distance))
    distances.sort(key = lambda tup: tup[1])
    neighbors = []
    for i in range(k):
        neighbors.append(distances[i][0])
    return neighbors

### Predecir las etiquetas usando k-nn.
#### Summary:
      Se obtiene la lista de los k-vecinos más cercanos, y se almacena el valor de
      la etiqueta en la lista "output_labels". Posteriormente se calcula el valor 
      promedio de las etiquetas y se almacena en la variable "prediction" y se retorna.

#### Args: 
      train(pyspark.sql.dataframe.DataFrame): Recibe el dataframe de entrenamiento
      test_row(pyspark.sql.types.Row): Recibe una instancia o renglón del dataset
      k(int): Número de k-vecinos que se desean obtener

In [8]:
def predict_classification(train, test_row, k):
    neighbors = get_neighbors(train, test_row, k)
    output_labels = [row[0] for row in neighbors]
    prediction = max(set(output_labels), key=output_labels.count)
    return prediction

### Clacular el porcentaje de exactitud.
#### Summary:
      Esta función calcula el porcentaje de exactitud del uso de k-NN, comparando
      las etiquetas reales de las instancias del dataset de entrenamiento y las
      etiquetas obtenidas mediante la predicción usando k-NN.
#### Args: 
      real_labels(pyspark.sql.dataframe.DataFrame): Recibe el dataframe de test que contiene los
                                                    valores reales de las etiquetas
      predicted(list): Lista con las etiquetas obtenidas mediante K-NN

In [9]:
def accuracy(real_labels, predicted):
    correct = 0
    total_rows = real_labels.count() + 1
    for i in range(1, total_rows):
        if(real_labels.head(i)[0][0] == predicted[i - 1]):
            correct += 1
    print("Correct labels: ", correct, 'of', (total_rows - 1))
    accuracy = correct / float(total_rows - 1)
    return accuracy

### Crear la función que calcule los vecinos más cercanos.
#### Summary:
      Se asignan los parámetros para calcular los k-vecinos más cercanos y hacer predicciones
      de las etiquetas a las que pertenecen, calculando la distancia entre las columnas de cada
      uno de los renglones del dataframe de "test" y el de "train", comparando las 
      reales con las otenidas por el clasificador y, finalmente, dado el porcentaje de exactitud obtenido. 
#### Args: 
      train(pyspark.sql.dataframe.DataFrame): Recibe el dataframe de entrenamiento
      test_row(pyspark.sql.dataframe.DataFrame): Recibe el dataframe de test
      k(int): Número de k-vecinos que se desean obtener

In [10]:
def k_nearest_neighbors(train, test, k):
    predictions = []
    total_test_rows = test.count() + 1
    for test_row in range(1, total_test_rows):
        output = predict_classification(train, test.head(test_row)[-1], k)
        predictions.append(output)
    mean_accuracy = accuracy(test, predictions)
    print("Mean accuracy: " + str(mean_accuracy))

### Se llama al método y se envían los parámetros

In [17]:
k_nearest_neighbors(train, test, k = 3)

Correct labels:  32 of 69
Mean accuracy: 0.463768115942029


## Prueba de cada método de K-NN  con el archivo Dimensión 5 x 76

In [11]:
# Prueba de la función euclidean_distance(), se mandan dos renglones del dataset total
length = data.count()               # Se obtiene el total de renglones en el dataset
for row in range(1, (length + 1)):
    distance = euclidean_distance(data.head(1)[-1], data.head(row)[-1])
    print("Dinstancia del renglon 1 con el", row, ":", distance)

Dinstancia del renglon 1 con el 1 : 0.0
Dinstancia del renglon 1 con el 2 : 3.1617478529944507
Dinstancia del renglon 1 con el 3 : 2.0356611659021397
Dinstancia del renglon 1 con el 4 : 1.4404646590152195
Dinstancia del renglon 1 con el 5 : 1.7914667218708056
Dinstancia del renglon 1 con el 6 : 2.8395962239571246
Dinstancia del renglon 1 con el 7 : 2.4695692287281386
Dinstancia del renglon 1 con el 8 : 3.161747853004572
Dinstancia del renglon 1 con el 9 : 3.3159346823953433
Dinstancia del renglon 1 con el 10 : 4.0780181152332435


In [12]:
# Prueba de la función get_neighbors(), se envían como args. el dataframe, un renglón de este y el número de vecinos.
length = data.count() + 1
neighbors = get_neighbors(data, data.head(1)[-1], k=3)
for neighbor in neighbors:
    print(neighbor)

Row(label=0.0, features=SparseVector(80, {3: 0.0539, 4: 0.0828, 5: 0.1176, 10: 0.2857, 15: 0.2, 16: 0.6557, 17: 0.7074, 18: 0.2835, 20: 0.2857, 21: 0.006, 23: 1.0, 27: 1.0, 35: 1.0, 43: 1.0, 53: 1.0, 55: 1.0, 61: 1.0, 63: 1.0, 65: 1.0, 67: 1.0, 69: 1.0, 71: 1.0, 73: 1.0, 75: 1.0}))
Row(label=0.0, features=SparseVector(80, {3: 0.0539, 4: 0.0828, 5: 0.1176, 10: 0.2857, 15: 0.1, 16: 0.6168, 17: 0.8055, 18: 0.3137, 20: 0.1429, 21: 0.1429, 23: 1.0, 24: 1.0, 27: 1.0, 32: 0.0556, 40: 0.1, 53: 1.0, 55: 1.0, 61: 1.0, 63: 1.0, 65: 1.0, 67: 1.0, 69: 1.0, 71: 1.0, 73: 1.0, 75: 1.0}))
Row(label=0.0, features=SparseVector(80, {1: 1.0, 3: 0.0622, 4: 0.0966, 5: 0.1176, 10: 0.2857, 16: 0.6339, 17: 0.788, 18: 0.2897, 20: 0.2857, 21: 0.0119, 23: 1.0, 24: 1.0, 27: 1.0, 32: 0.0556, 40: 0.1, 53: 1.0, 55: 1.0, 61: 1.0, 63: 1.0, 65: 1.0, 67: 1.0, 69: 1.0, 71: 1.0, 73: 1.0, 75: 1.0}))


In [13]:
# Prueba de la función get_neighbors(), se envían como args. el dataframe, un renglón de este y el número de vecinos y n es igual al número de renglón.
n = 1
print("Row class:", data.head(n)[-1][0] ) # CLase/Label
prediction = predict_classification(data, data.head(n)[-1], k=3)
print('Expected label: %d, Got: %d.' % (data.head(n)[-1][0], prediction))

Row class: 0.0
Expected label: 0, Got: 0.


# Segmento de pruebas locas

In [ ]:
# Se asignan los RDD para el posterior procesamiento
rdd_train = train.rdd
rdd_test = test.rdd

In [ ]:
# Se utilizan dos array de numpy para alamacenar las instancias del set de entrenamiento y procesar con un RDD y el segundo array almacena las etiquetas. 
#train_array = np.array(train.select('features').collect(), dtype=float)
#train_array_labels = np.array(train.select('label').collect(), dtype=float)

In [ ]:
# Etiquteas de las instancias del conjunto de test. 
test_array_labels = np.array(test.select('label').collect(), dtype=float)

In [ ]:
print('RDD de entrenamiento: ' + str(rdd_train.count()))
print('RDD de test: ' + str(rdd_test.count()))

In [ ]:
# Metodo que guarda cada renglon en un archivo .svm
def save_file(data):
    file = open('../data/url_svmlight/Distancia_euclideana_100_x_500000.svm', 'a')
    file.write(data)
    file.close()

In [ ]:
"""
[summary:
    Método para calcular la distancia euclídea entre cada una de las 
    columnas del conjunto de test respecto a las columnas del conjunto
    de entrenamiento.
]

Args:
    instance ([pyspark.sql.types.Row]): [
        Recibe cada una de las instancias que hay en el dataset
    ]
"""
def euclidean_distance(instance):
    distance = 0
    instance_distance = ''
    for row in range(len(train_array)):
        instance_distance += str(train_array_labels[row][0]) + ' '
        for column in range(len(instance[1])):
            distance = pow(train_array[row][0][column] - instance.features[column], 2)
            distance = math.sqrt(distance)
            # instance_distance += str(column + 1) +':' + str(distance) + ' ' # -> Si quisiera poner los indices de cada caracteristica.
            instance_distance += str(distance) + ' '
        instance_distance += '\n'
    save_file(instance_distance)

In [ ]:
# Ejecuta el método que calcula la distancia euclídea entre los puntos euclidean_distance()
test.foreach(euclidean_distance)

In [ ]:
rdd_samp1 = sc.textFile('../data/url_svmlight/arch_prb.svm')
rdd_samp2 = sc.textFile('../data/url_svmlight/arch_prb1.svm')
rdd_samp3 = sc.textFile('../data/url_svmlight/arch_prb2.svm')

In [ ]:
five_nearest1 = rdd_samp1.takeOrdered(5)
five_nearest2 = rdd_samp2.takeOrdered(5)
five_nearest3 = rdd_samp3.takeOrdered(5)

In [ ]:
def class_average(five_nearest):
    mean = 0
    for i in range(5):
        mean += float(five_nearest[i][0])
    mean = mean / 5
    if(mean > 0.5):
        print('Clase K-NN: 1')
        return 1
    else:
        print('Clase K-NN: 0')
        return 0

In [ ]:
def accuracy():
    lista = [five_nearest1, five_nearest2, five_nearest3]
    accuracy = 0.0
    for i in range(len(test_array_labels)):
        if(test_array_labels[i][0] == class_average(lista[i])):
            accuracy += 1
        print('Clase Real: ' + str(test_array_labels[i][0]))
        print('\n')
    accuracy = accuracy / len(test_array_labels)
    print('Accuracy: ' + str(accuracy))

In [ ]:
accuracy()

In [ ]:
lista = [five_nearest1, five_nearest2, five_nearest3]

In [ ]:
# listaclass_average

In [ ]:
# five_nearest[0][0] # Clase

In [ ]:
# rdd_prueba2 = sc.textFile('../data/url_svmlight/Distancia_euclideana_5_x_76.svm')

In [ ]:
five_nearest2 = rdd_prueba.takeOrdered(5)
type(five_nearest2)

In [ ]:
five_nearest2[0][0]

In [ ]:
pwd